In [11]:
import pandas as pd
from my_rymscraper.rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [12]:
mt = pd.read_csv("metacritic_data.csv")

In [13]:
mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"];
lower_alnum = lambda s: "".join(x for x in s.lower() if x.isalnum())
mt["id"] = mt["Singer"].apply(lower_alnum) + mt["Release_Year"].apply(str) + mt["Album_Name"].apply(lambda s: lower_alnum(s)[0:3] + lower_alnum(s)[-4:-1])
tags = zip(mt["tag"],mt["id"])
# tags = ["Common - A Beautiful Revolution, Vol. 2"]
df = pd.DataFrame()

In [14]:
max_iter = 1
iter = 0
while iter < max_iter:
  new_tags = []
  for tag, uid in tags:
    try:
      album_infos = network.get_album_infos(name=tag)
      if not album_infos:
        print("❌ ", tag)
        new_tags.append((tag,uid))
      else:
        print("✅ ", tag)
        album_infos["id"] = uid
        df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
    # except IndexError:
    except (IndexError, AttributeError):
      new_tags.append((tag,uid))
      print("❌ ", tag)

  tags = new_tags
  iter += 1

✅  Noname - Room 25
✅  Rolo Tomassi - Time Will Die and Love Will Bury It
✅  Architects - Holy Hell
✅  Robyn - Honey
✅  Yves Tumor - Safe In The Hands Of Love
✅  Kacey Musgraves - Golden Hour
✅  Christine and the Queens - Chris
✅  Rival Consoles - Persona
✅  Jeff Rosenstock - POST-
✅  Idles - Joy As an Act of Resistance
✅  Behemoth - I Loved You at Your Darkest
✅  Marianne Faithfull - Negative Capability
✅  Janelle Monáe - Dirty Computer
✅  Car Seat Headrest - Twin Fantasy
✅  Hookworms - Microshift


Rate-limit detected. Restarting browser.


✅  Alela Diane - Cusp
✅  Objekt - Cocoon Crush
✅  Daughters - You Won't Get What You Want
✅  Mitski - Be the Cowboy
✅  serpentwithfeet - Soil
✅  U.S. Girls - In a Poem Unlimited
✅  Young Fathers - Cocoa Sugar
✅  Amen Dunes - Freedom
✅  Kali Uchis - Isolation
✅  Bruce Springsteen - Springsteen on Broadway
✅  SOPHIE - Oil Of Every Pearl's Un-Insides
✅  Ashley Monroe - Sparrow
✅  Tracyanne & Danny - Tracyanne & Danny
✅  Hot Snakes - Jericho Sirens
✅  Angélique Kidjo - Remain In Light


Rate-limit detected. Restarting browser.


✅  Kamasi Washington - Heaven & Earth
✅  Mélissa Laveaux - Radyo Siwèl
✅  Low - Double Negative
✅  Amanda Shires - To the Sunset
✅  Mount Eerie - (After) [Live]
✅  Pusha T - DAYTONA
✅  Jonny Greenwood - Phantom Thread [Original Motion Picture Soundtrack]
✅  Earl Sweatshirt - Some Rap Songs
✅  Screaming Females - All at Once
✅  DJ Koze - Knock Knock
✅  Alva Noto - Glass
✅  BROCKHAMPTON - Iridescence
✅  Deafheaven - Ordinary Corrupt Human Love
✅  Marie Davidson - Working Class Woman
✅  Ezra Furman - Transangelic Exodus


...

Rate-limit detected. Restarting browser.


✅  Willie Nelson - Last Man Standing
✅  The Necks - Body
✅  CupcakKe - Ephorize
✅  Rolling Blackouts Coastal Fever - Hope Downs
✅  Proc Fiskal - Insula
✅  Cardi B - Invasion of Privacy
✅  Jon Hopkins - Singularity
✅  Sleep - The Sciences
✅  Stephen Malkmus & the Jicks - Sparkle Hard
✅  The Breeders - All Nerve
✅  Jon Hassell - Listening to Pictures (Pentimento Volume I)
✅  Miya Folick - Premonitions
✅  Blood Orange - Negro Swan
✅  Eleanor Friedberger - Rebound
✅  Chilly Gonzales - Solo Piano III


Rate-limit detected. Restarting browser.


✅  Laurel Halo - Raw Silk Uncut Wood
✅  Anna Calvi - Hunter
✅  Jean Grae - Everything's Fine
✅  Daniel Avery - Song for Alpha


KeyboardInterrupt: 

In [ ]:
# new_tags_2 = []
# found = 0
# for tag2, uid2 in tags:
#     if uid2 == "fayewebster2021ikn":
#         found = 1
#     if not found:
#         continue
#     new_tags_2.append((tag2,uid2))

In [ ]:
xdf = df.reset_index()
xdf = df[['Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages']]
xdf.reset_index()
xdf.describe()

,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages
count,543,543,543,542,543,543,450,543,543,24
unique,542,525,118,542,537,481,5,543,531,17
top,Palomino,Lambchop,8 April 2022,3.37\n \n/ 5.00.5\n\n ...,"Singer-Songwriter, Contemporary Folk",,English,"[Back of My Hand, Sand Angel, Stay in the Car,...","[#7a7a7a, #cccccc]","English, Spanish"
freq,2,2,15,1,3,28,446,1,13,7


In [ ]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/var/folders/qh/04ndk5cd4w10t0bhkqvrbnh40000gn/T/ipykernel_27015/1331460494.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]


In [ ]:
xdf = xdf[['Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme']]
xdf.describe()

,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme
count,1488,1488,1488,1486,1488,1488,1187,1488,1488
unique,1479,1311,247,1474,1435,1293,31,1475,1439
top,Girl,King Gizzard & The Lizard Wizard,2 March 2018,3.30\n \n/ 5.00.5\n\n ...,,,English,[],"[#7a7a7a, #cccccc]"
freq,2,5,21,2,5,113,1130,13,49


In [ ]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [ ]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [ ]:
xdf

In [ ]:
xdf.to_csv("rym_new.csv")